In [7]:
import polars as pl
import json

import sys
sys.path.append('..')
from utils import utils as u

In [8]:
# read csv with polars module
df = json.loads(pl.read_csv("../datasets/TLN-definitions-23.tsv", separator='\t').write_json())

data = {}

In [9]:
for col in df['columns']:
    if col['name'] != '1':
        data[col['name']] = {}
        count = 1
        for v in col['values']:
            data[col['name']][count] = u.noise_reduction_en(v)
            count += 1



In [10]:
def compute_similarity(definition_1_tokens, definition_2_tokens):

    min_len = 0

    if len(definition_1_tokens) > len(definition_2_tokens):
        min_len = len(definition_2_tokens)
    else:
        min_len = len(definition_1_tokens)

    return (len(set(definition_1_tokens).intersection(set(definition_2_tokens))) / min_len)


In [11]:
results = {}
final = {}
for word in data:
    results[word] = {}
    for id in data[word]:
        for id_2 in data[word]:
            if id != id_2 and ((id, id_2), (id_2, id)) not in results[word]:
                results[word][((id, id_2), (id_2, id))] = compute_similarity(
                    data[word][id], data[word][id_2])

    final[word] = round((sum(results[word].values()) / \
        len(results[word])if len(results[word]) > 0 else 0), 2)


In [12]:


data = {
    "": ["Generico", "Specifico"],
    "Astratto": ["pain: "+str(final['pain']), "blurriness: "+str(final['blurriness'])],
    "Concreto": ["door: "+str(final['door']), "ladybug: "+str(final['ladybug'])]
}

# Crea un DataFrame con i dati utilizzando la libreria polars
table = pl.DataFrame(data)

# Stampa la tabella utilizzando la funzione print()
print(table)


shape: (2, 3)
┌───────────┬──────────────────┬───────────────┐
│           ┆ Astratto         ┆ Concreto      │
│ ---       ┆ ---              ┆ ---           │
│ str       ┆ str              ┆ str           │
╞═══════════╪══════════════════╪═══════════════╡
│ Generico  ┆ pain: 0.17       ┆ door: 0.21    │
│ Specifico ┆ blurriness: 0.08 ┆ ladybug: 0.49 │
└───────────┴──────────────────┴───────────────┘
